# **About**

|Section|Details|
|---|---|
|Script|rs3-request-data|
|Description|rs3-request-data is used to retrieve historical price data for items in Runescape 3. This data will be transformed to predict future item prices.|
|Author|Andrew Yang|

# **Setup**

In [326]:
import requests # use to retrieve data from API
import math
import pandas as pd
import datetime
#from datetime import datetime, timedelta, UTC
#import os

In [327]:
# Global var for debugging
show_debug = True

# Global var for integrating social media information into the model
integrate_social = False

# Global vars for Weird Gloop API
game_base = "rs" # API can have two options: rs (Runescape 3) or osrs (Old School Runescape)
data_filter = "all" # API has three options: all (all price data), last90d (last 90 days), and sample

# Determine which item category we are interested in.
# See the following link for all item categories (https://runescape.wiki/w/Application_programming_interface#category).
item_category = 13 

# **Data Gathering**

##  Retrieve items by category

### Get item categories

In [328]:
# See the following link for a description of all item categories (https://runescape.wiki/w/Application_programming_interface#category).
# i.e. 0 = Miscellaneous; 7 = Costumes; 13 = Herblore materials 
category_ids = range(0, 44, 1)
print(f"All category ids: {list(category_ids)}")

All category ids: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]


### Retrieve items in category

In [329]:
# Grabs a list of dictionaries that show how many items are under each "alpha" (i.e. character)
request_category_base = "https://services.runescape.com/m=itemdb_rs/api/catalogue/category.json?"
r_category = requests.get(request_category_base, params = {"category": item_category})
print(r_category.status_code)

200


In [330]:
category_alpha_dict = r_category.json()["alpha"]

print(f"Item alpha dict: ")
display(category_alpha_dict)

Item alpha dict: 


[{'letter': '#', 'items': 0},
 {'letter': 'a', 'items': 6},
 {'letter': 'b', 'items': 7},
 {'letter': 'c', 'items': 32},
 {'letter': 'd', 'items': 5},
 {'letter': 'e', 'items': 1},
 {'letter': 'f', 'items': 2},
 {'letter': 'g', 'items': 29},
 {'letter': 'h', 'items': 2},
 {'letter': 'i', 'items': 2},
 {'letter': 'j', 'items': 3},
 {'letter': 'k', 'items': 2},
 {'letter': 'l', 'items': 6},
 {'letter': 'm', 'items': 13},
 {'letter': 'n', 'items': 1},
 {'letter': 'o', 'items': 0},
 {'letter': 'p', 'items': 6},
 {'letter': 'q', 'items': 0},
 {'letter': 'r', 'items': 5},
 {'letter': 's', 'items': 12},
 {'letter': 't', 'items': 5},
 {'letter': 'u', 'items': 2},
 {'letter': 'v', 'items': 3},
 {'letter': 'w', 'items': 6},
 {'letter': 'x', 'items': 0},
 {'letter': 'y', 'items': 2},
 {'letter': 'z', 'items': 1}]

In [331]:
# Define API endpoint for getting item info.
request_items_base = "https://services.runescape.com/m=itemdb_rs/api/catalogue/items.json?"

def get_item_details(req_category, req_alpha, req_page):
    r_items = requests.get(request_items_base, params = {"category": req_category, "alpha": req_alpha, "page": req_page})
    if show_debug:
        print(f"Status of item category|alpha|page ({req_category}|{req_alpha}|{req_page}): {r_items.status_code}")
    return r_items

In [332]:
item_dict_list = {}

# Add entries to a dictionary linking an item id to item name.
def get_item_id(item_request):
    for item in item_request.json()["items"]:
        item_dict_list[item["id"]] = item["name"]

In [333]:
for a in category_alpha_dict:
    if a["items"] <= 0: 
        continue
    
    req_requests =  math.ceil(a["items"] / 12) # Determines the # of times to request the API for all items.
    for page in range(1, req_requests + 1, 1):
        get_item_id(get_item_details(item_category, a["letter"], page))

Status of item category|alpha|page (13|a|1): 200
Status of item category|alpha|page (13|b|1): 200
Status of item category|alpha|page (13|c|1): 200
Status of item category|alpha|page (13|c|2): 200
Status of item category|alpha|page (13|c|3): 200
Status of item category|alpha|page (13|d|1): 200
Status of item category|alpha|page (13|e|1): 200
Status of item category|alpha|page (13|f|1): 200
Status of item category|alpha|page (13|g|1): 200
Status of item category|alpha|page (13|g|2): 200
Status of item category|alpha|page (13|g|3): 200
Status of item category|alpha|page (13|h|1): 200
Status of item category|alpha|page (13|i|1): 200
Status of item category|alpha|page (13|j|1): 200
Status of item category|alpha|page (13|k|1): 200
Status of item category|alpha|page (13|l|1): 200
Status of item category|alpha|page (13|m|1): 200
Status of item category|alpha|page (13|m|2): 200
Status of item category|alpha|page (13|n|1): 200
Status of item category|alpha|page (13|p|1): 200
Status of item categ

In [334]:
# print(f"Item id dictionary ({len(item_dict_list)} entries):")
# display(item_dict_list)

##  Retrieving item GE prices from API

In [335]:
# The RS3 Wiki uses Weird Gloop for their API, and includes support for retrieving their stored Grand Exchange data.
# We can avoid request limitations with Jagex's own Grand Exchange API, and get all historical data.
request_prices_base = f"https://api.weirdgloop.org/exchange/history/{game_base}/{data_filter}"
print(f"Historical GE prices request endpoint: {request_prices_base}")

Historical GE prices request endpoint: https://api.weirdgloop.org/exchange/history/rs/all


In [336]:
def get_historical_prices(item_filter):
    r_prices = requests.get(request_prices_base, params = {"id": item_filter})
    if show_debug:
        print(f"Status of item {item_filter} ({item_dict_list[item_filter]}):{r_prices.status_code}")
    return r_prices.json()[f"{item_filter}"]

In [337]:
ge_prices = []
for item_id_entry in item_dict_list.keys():
    ge_prices.extend(get_historical_prices(item_id_entry))

Status of item 52937 (Abyssal flesh):200
Status of item 39067 (Adrenaline crystal):200
Status of item 48241 (Arbuck potion (unf)):200
Status of item 592 (Ashes):200
Status of item 103 (Avantoe potion (unf)):200
Status of item 48578 (Avocado):200
Status of item 48925 (Beak snot):200
Status of item 37973 (Bloodweed potion (unf)):200
Status of item 243 (Blue dragon scale):200
Status of item 48961 (Bomb vial):200
Status of item 48926 (Bottled dinosaur roar):200
Status of item 4456 (Bowl of hot water):200
Status of item 43993 (Bull horns):200
Status of item 6016 (Cactus spine):200
Status of item 107 (Cadantine potion (unf)):200
Status of item 48586 (Carambola):200
Status of item 11326 (Caviar):200
Status of item 43973 (Chinchompa residue):200
Status of item 48584 (Ciku):200
Status of item 48211 (Clean arbuck):200
Status of item 261 (Clean avantoe):200
Status of item 37953 (Clean bloodweed):200
Status of item 265 (Clean cadantine):200
Status of item 267 (Clean dwarf weed):200
Status of item 

In [338]:
#display(ge_prices)

## Consolidate data into dataframe

In [339]:
# Convert our price list of dictionaries into a Pandas Dataframe.
ge_df = pd.DataFrame(ge_prices)

In [340]:
#display(ge_df)

# Data supplementation

In [341]:
# Checking how many null values are there - will most likely be within the volume.
ge_df.isna().sum() / len(ge_df)

id           0.000000
price        0.000000
volume       0.849735
timestamp    0.000000
dtype: float64

## Improve human readability

In [342]:
# Convert unix timestamp to date.
def unix_to_date(ts):
    return datetime.datetime.fromtimestamp(ts/1000, datetime.UTC).strftime('%Y-%m-%d')

In [343]:
# Determine if a unix timestamp represents a weekend or weekday.
def unix_is_weekday(ts):
    return 1 if datetime.datetime.fromtimestamp(ts/1000, datetime.UTC).weekday() < 5 else 0

In [344]:
# Adds name of item based on item id.
def id_to_name(item_id):
    return item_dict_list[int(item_id)]

In [345]:
# Make human-readable date, item name, and whether or not said date is a weekday.
ge_df["date"] = ge_df['timestamp'].map(unix_to_date)
ge_df["weekday"] = ge_df['timestamp'].map(unix_is_weekday)
ge_df["name"] = ge_df['id'].map(id_to_name)

## Add differenced time series

In [346]:
# Get differenced data by 1 day, 1 week, 2 weeks, and ~ 1 month. Great for time series.
ge_df["diff_1_day"] = ge_df.groupby("id")["price"].diff(1)
ge_df["diff_7_day"] = ge_df.groupby("id")["price"].diff(7)
ge_df["diff_14_day"] = ge_df.groupby("id")["price"].diff(14)
ge_df["diff_30_day"] = ge_df.groupby("id")["price"].diff(30)

## Add moving average time series

In [347]:
# Get 1 week, 2 week, and ~1 month moving average.
ge_df["ma_7_day"] = ge_df.groupby("name")["price"].rolling(7, 1).mean().reset_index(drop=True)
ge_df["ma_14_day"] = ge_df.groupby("name")["price"].rolling(14, 1).mean().reset_index(drop=True)
ge_df["ma_30_day"] = ge_df.groupby("name")["price"].rolling(30, 1).mean().reset_index(drop=True)

## Clean price time series

In [348]:
# Get the difference between 1 week, 2 week, and ~1 month moving average for each day.
ge_df["diff_ma_7_day"] = ge_df.groupby("id")["ma_7_day"].diff(1)
ge_df["diff_ma_14_day"] = ge_df.groupby("id")["ma_14_day"].diff(1)
ge_df["diff_ma_30_day"] = ge_df.groupby("id")["ma_30_day"].diff(1)

In [349]:
# Create the following conditions to filter ge_df:
# cond1 - If the difference in moving average at a dat for 1 week, 2 weeks, and 1 month are = 0, the date's price is likely the initial Jagex set price.
# cond2 - Similarly, remove rows with null values in difference in moving average.
# cond3 - Finally, as a precaution, remove null rows for price differences at 1 day, 1 week, 2 weeks, and 1 month.

cond1 = (ge_df["diff_ma_7_day"] != 0) & (ge_df["diff_ma_14_day"] != 0) & (ge_df["diff_ma_30_day"] != 0)
cond2 = (ge_df["diff_ma_7_day"].notnull()) & (ge_df["diff_ma_14_day"].notnull()) & (ge_df["diff_ma_30_day"].notnull())
cond3 = (ge_df["diff_1_day"].notnull()) & (ge_df["diff_7_day"].notnull()) & (ge_df["diff_14_day"].notnull())& (ge_df["diff_30_day"].notnull())

# Filter original ge prices dataframe based on the above conditions to get clean values.
ge_enriched_df = ge_df[cond1 & cond2 & cond3].copy().reset_index()
display(ge_enriched_df.sample(10, random_state = 123))

,index,id,price,volume,timestamp,date,weekday,name,diff_1_day,diff_7_day,diff_14_day,diff_30_day,ma_7_day,ma_14_day,ma_30_day,diff_ma_7_day,diff_ma_14_day,diff_ma_30_day
299418,340533,3049,5353,NaN,1501804800000,2017-08-04,1,Grimy toadflax,-71.0,-13.0,110.0,166.0,5442.285714,5397.714286,5231.866667,-1.857143,7.857143,5.533333
339131,404898,101,3356,599910.0,1691660544000,2023-08-10,1,Irit potion (unf),0.0,187.0,620.0,1053.0,3284.142857,3112.571429,2793.200000,26.714286,44.285714,35.100000
129216,144535,12172,4608,NaN,1537056000000,2018-09-16,0,Clean spirit weed,0.0,-479.0,-682.0,-203.0,4813.285714,4960.857143,5068.733333,-68.428571,-48.714286,-6.766667
348095,423003,10111,1384,NaN,1376870400000,2013-08-19,1,Kebbit teeth dust,0.0,-5.0,-2.0,-15.0,1385.428571,1387.214286,1384.400000,-0.714286,-0.142857,-0.500000
467479,624497,1939,425,NaN,1405296000000,2014-07-14,1,Swamp tar,-9.0,-13.0,16.0,-20.0,449.142857,440.071429,428.566667,-1.857143,1.142857,-0.666667
96326,110773,259,925,180457.0,1674902136000,2023-01-28,0,Clean irit,0.0,-2.0,-89.0,-320.0,918.142857,943.428571,1047.900000,-0.285714,-6.357143,-10.666667
178904,209936,9735,302,NaN,1317254400000,2011-09-29,1,Desert goat horn,-15.0,-89.0,-174.0,-246.0,348.285714,390.857143,449.033333,-12.714286,-12.428571,-8.200000
369969,455921,11328,407,1281.0,1676016730000,2023-02-10,1,Leaping trout,0.0,-1.0,-1.0,-1.0,407.285714,407.642857,407.833333,-0.142857,-0.071429,-0.033333
72975,86854,267,2552,4400000.0,1541203200000,2018-11-03,0,Clean dwarf weed,122.0,634.0,599.0,371.0,2216.571429,2079.142857,2060.500000,90.571429,42.785714,12.366667
415239,554512,23191,4452,NaN,1586217600000,2020-04-07,1,Potion flask,-164.0,-1064.0,-634.0,-244.0,4995.142857,5205.642857,4994.700000,-152.000000,-45.285714,-8.133333


## Get social media sourced information

In [350]:
# The RS3 Wiki uses Weird Gloop for their API; this endpoint gets all social media information.
request_socials_base = f"https://api.weirdgloop.org/runescape/social"
halt = False # Used in while loop - API response includes if there are additional pages left.
max_iter = 100 # Failsafe.

In [351]:
page = 1
social_dict_list = []

while halt != True and page <= max_iter: # As long as we haven't halted the process and page less than the max allowed
    r_social = requests.get(request_socials_base, params = {"page": page}) # Request social media info from API.
    
    if show_debug:
        print(f"Status of page {page}: {r_social.status_code}")
    page += 1

    if r_social.json()["pagination"]["has_more"] != True: # If the response tells use there's no more pages, halt the loop.
        halt = True

    social_dict_list.extend(r_social.json()["data"]) # Add dictionaries to our list.

Status of page 1: 200
Status of page 2: 200
Status of page 3: 200
Status of page 4: 200
Status of page 5: 200
Status of page 6: 200
Status of page 7: 200
Status of page 8: 200
Status of page 9: 200
Status of page 10: 200
Status of page 11: 200
Status of page 12: 200
Status of page 13: 200
Status of page 14: 200
Status of page 15: 200
Status of page 16: 200
Status of page 17: 200
Status of page 18: 200
Status of page 19: 200
Status of page 20: 200
Status of page 21: 200
Status of page 22: 200
Status of page 23: 200
Status of page 24: 200
Status of page 25: 200
Status of page 26: 200
Status of page 27: 200
Status of page 28: 200
Status of page 29: 200
Status of page 30: 200
Status of page 31: 200
Status of page 32: 200
Status of page 33: 200
Status of page 34: 200
Status of page 35: 200
Status of page 36: 200
Status of page 37: 200
Status of page 38: 200
Status of page 39: 200
Status of page 40: 200
Status of page 41: 200
Status of page 42: 200
Status of page 43: 200
Status of page 44: 2

In [352]:
# Get social media list of dicts into a dataframe.
social_df = pd.DataFrame(social_dict_list)

In [353]:
# Convert odd datetime format to string, and get date.
def datetime_to_string(dt):
    return str(dt)[:10]

In [354]:
# Create string version of date to link social media info to specific dates.
social_df["date"] = social_df["dateAdded"].map(datetime_to_string)

In [355]:
# Enrich social media history dataframe based on title of media item:
#   Launch usually indicates a new release.
#   Bosses are big drop sources of items, and may affect our items of interest.
#   Quests show info about an upcoming Runescape quest. These quests may unlock new things which require our items of interest.
#   Event usually indicates a new upcoming events.
#   Double XP tells players when the next Double XP is coming up, and is a known market mover.
#   Update is more general, but can include information on changes for any of the above info... or something irrelevant.
social_df["launch_update"] = social_df["title"].apply(lambda x: "Launch" in x if x is not None else False)
social_df["boss_update"] = social_df["title"].apply(lambda x: "Boss" in x if x is not None else False)
social_df["quest_update"] = social_df["title"].apply(lambda x: "Quest" in x if x is not None else False)
social_df["event_update"] = social_df["title"].apply(lambda x: "Event" in x if x is not None else False)
social_df["dxp_update"] = social_df["title"].apply(lambda x: "Double XP" in x if x is not None else False)
social_df["general_update"] = social_df["title"].apply(lambda x: "Update" in x if x is not None else False)

In [356]:
ge_dates = sorted(ge_enriched_df.date.unique())
print(f"Earliest GE date: {ge_dates[0]}; Most recent GE date: {ge_dates[-1]}.")

earliest_update_date = social_df.date.min()
recent_update_date = social_df.date.max()
print(f"Earliest update: {earliest_update_date}; Most recent update: {recent_update_date}.")

Earliest GE date: 2008-06-20; Most recent GE date: 2024-07-28.
Earliest update: 2020-02-14; Most recent update: 2024-07-22.


In [357]:
# Get range of 
max_update_date = max(recent_update_date, ge_dates[-1])
min_update_date = max(earliest_update_date, ge_dates[0])
print(f"Earliest simulated date: {min_update_date}; Most recent simulated date: {max_update_date}.")

sim_min_date = datetime.datetime.strptime(min_update_date, '%Y-%m-%d')
sim_max_date = datetime.datetime.strptime(max_update_date, '%Y-%m-%d')

Earliest simulated date: 2020-02-14; Most recent simulated date: 2024-07-28.


In [358]:
# Sets iteration range.
range_days = pd.date_range(sim_min_date, sim_max_date).to_list()
print(f"Iterate over {len(range_days)} days.")

update_concat_list = []
for d in range_days:
    update_concat_list.append([d.strftime('%Y-%m-%d'), False, False, False, False, False, False])

print(update_concat_list[0])

Iterate over 1627 days.
['2020-02-14', False, False, False, False, False, False]


In [359]:
social_temp_df = social_df[['date', 'launch_update', 'quest_update', 'event_update', 'dxp_update', 
                            'general_update', 'boss_update']]
social_append_df = pd.DataFrame(update_concat_list, columns = social_temp_df.columns)
social_enriched_df = pd.concat([social_temp_df, social_append_df], axis = 0)
display(social_enriched_df.sample(10, random_state = 123))

,date,launch_update,quest_update,event_update,dxp_update,general_update,boss_update
728,2022-02-11,False,False,False,False,False,False
267,2020-11-07,False,False,False,False,False,False
1451,2024-02-04,False,False,False,False,False,False
716,2022-01-30,False,False,False,False,False,False
89,2020-05-13,False,False,False,False,False,False
589,2021-09-25,False,False,False,False,False,False
819,2022-05-13,False,False,False,False,False,False
63,2020-04-17,False,False,False,False,False,False
289,2022-05-02,False,False,False,False,False,False
563,2020-09-01,False,False,False,False,False,False


In [360]:
# Aggregate by date to determine if an update occured on or around a specific date.
social_agg_df = social_enriched_df.groupby(["date"], as_index=False).any()
#display(social_agg_df)

,date,launch_update,quest_update,event_update,dxp_update,general_update,boss_update
0,2020-02-14,False,False,False,False,False,False
1,2020-02-15,False,False,False,False,False,False
2,2020-02-16,False,False,False,False,False,False
3,2020-02-17,False,False,False,False,False,False
4,2020-02-18,False,False,False,False,False,False
...,...,...,...,...,...,...,...
1622,2024-07-24,False,False,False,False,False,False
1623,2024-07-25,False,False,False,False,False,False
1624,2024-07-26,False,False,False,False,False,False
1625,2024-07-27,False,False,False,False,False,False


In [361]:
# Calculate moving averages for 7, 14, and 30 days and put into dataframes.
# FUTURE UPDATE - experiment with different window weight methods (like exponential or gaussian), to represent decaying influence of update.
social_agg_df_7_ma = social_agg_df[social_agg_df.columns[social_agg_df.columns!='date']].rolling(7, 1).mean()
social_agg_df_14_ma = social_agg_df[social_agg_df.columns[social_agg_df.columns!='date']].rolling(14, 1).mean()
social_agg_df_30_ma = social_agg_df[social_agg_df.columns[social_agg_df.columns!='date']].rolling(30, 1).mean()

In [362]:
# Rename columns for clarity.
social_agg_df_7_ma.columns = [f"{c}_7_ma" for c in social_agg_df_7_ma.columns.to_list()]
social_agg_df_14_ma.columns = [f"{c}_14_ma" for c in social_agg_df_14_ma.columns.to_list()]
social_agg_df_30_ma.columns = [f"{c}_30_ma" for c in social_agg_df_30_ma.columns.to_list()]

In [363]:
# Join together dataframes of original data + moving averages.
social_agg_final_df = pd.concat([social_agg_df, social_agg_df_7_ma, social_agg_df_14_ma, social_agg_df_30_ma], axis=1)
display(social_agg_final_df)

,date,launch_update,quest_update,event_update,dxp_update,general_update,boss_update,launch_update_7_ma,quest_update_7_ma,event_update_7_ma,...,event_update_14_ma,dxp_update_14_ma,general_update_14_ma,boss_update_14_ma,launch_update_30_ma,quest_update_30_ma,event_update_30_ma,dxp_update_30_ma,general_update_30_ma,boss_update_30_ma
0,2020-02-14,False,False,False,False,False,False,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
1,2020-02-15,False,False,False,False,False,False,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
2,2020-02-16,False,False,False,False,False,False,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
3,2020-02-17,False,False,False,False,False,False,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
4,2020-02-18,False,False,False,False,False,False,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1622,2024-07-24,False,False,False,False,False,False,0.142857,0.0,0.0,...,0.0,0.0,0.0,0.142857,0.033333,0.0,0.0,0.0,0.0,0.066667
1623,2024-07-25,False,False,False,False,False,False,0.142857,0.0,0.0,...,0.0,0.0,0.0,0.142857,0.033333,0.0,0.0,0.0,0.0,0.066667
1624,2024-07-26,False,False,False,False,False,False,0.142857,0.0,0.0,...,0.0,0.0,0.0,0.142857,0.033333,0.0,0.0,0.0,0.0,0.066667
1625,2024-07-27,False,False,False,False,False,False,0.142857,0.0,0.0,...,0.0,0.0,0.0,0.142857,0.033333,0.0,0.0,0.0,0.0,0.066667


## Join in social media info

In [364]:
# Determine final ge price data, based on whether or not to integrate social media.
ge_final_df = ge_enriched_df
if integrate_social:
    ge_final_df = ge_enriched_df.merge(social_agg_final_df, left_on='date', right_on='date', how = "left")

# View results

In [365]:
display(ge_final_df)

,index,id,price,volume,timestamp,date,weekday,name,diff_1_day,diff_7_day,diff_14_day,diff_30_day,ma_7_day,ma_14_day,ma_30_day,diff_ma_7_day,diff_ma_14_day,diff_ma_30_day
0,180,52937,1653,NaN,1646092800000,2022-03-01,1,Abyssal flesh,153.0,153.0,153.0,153.0,1521.857143,1510.928571,1505.100000,21.857143,10.928571,5.100000
1,181,52937,1736,NaN,1646179200000,2022-03-02,1,Abyssal flesh,83.0,236.0,236.0,236.0,1555.571429,1527.785714,1512.966667,33.714286,16.857143,7.866667
2,182,52937,1823,NaN,1646265600000,2022-03-03,1,Abyssal flesh,87.0,323.0,323.0,323.0,1601.714286,1550.857143,1523.733333,46.142857,23.071429,10.766667
3,183,52937,1915,NaN,1646352000000,2022-03-04,1,Abyssal flesh,92.0,415.0,415.0,415.0,1661.000000,1580.500000,1537.566667,59.285714,29.642857,13.833333
4,184,52937,2011,NaN,1646438400000,2022-03-05,0,Abyssal flesh,96.0,511.0,511.0,511.0,1734.000000,1617.000000,1554.600000,73.000000,36.500000,17.033333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550665,723621,43983,483,17274.0,1721804620000,2024-07-24,1,Zygomite fruit,0.0,-19.0,-44.0,-63.0,485.714286,499.000000,510.466667,-2.714286,-3.142857,-2.100000
550666,723622,43983,480,15194.0,1721928652000,2024-07-25,1,Zygomite fruit,-3.0,-22.0,-34.0,-66.0,482.571429,496.571429,508.266667,-3.142857,-2.428571,-2.200000
550667,723623,43983,480,11417.0,1721993510000,2024-07-26,1,Zygomite fruit,0.0,-3.0,-34.0,-39.0,482.142857,494.142857,506.966667,-0.428571,-2.428571,-1.300000
550668,723624,43983,480,21458.0,1722066488000,2024-07-27,0,Zygomite fruit,0.0,-3.0,-34.0,-39.0,481.714286,491.714286,505.666667,-0.428571,-2.428571,-1.300000


# Export data

In [366]:
# Create file based on date.
today = datetime.date.today().strftime('%Y-%m-%d')
print(f"Current date: {today}")

filename = f"ge-prices-{today}.csv"
print(f"Final filename: {filename}")

Current date: 2024-07-28
Final filename: ge-prices-2024-07-28.csv


In [367]:
# Export file.
#ge_final_df.to_csv(filename, index=False)  